# Real Estate Listing Agent

As I've previously worked in Real Estate, I wanted to explore use cases in real estate using GenAI. 

A simple use case I've considered is for an LLM to recognize key features in an image that would be useful for automatically labeling images for indexing so it can be searchable. I've seen too many images that are not adequately labeled. Does the listing have no carpet? I have to look at the images. Too often, listings are not easily searcheable across a number of attributes I would fine desirable. 

For this project, I started with the GenAI capability of Image Understanding. I would see how far I would get with the time I had. By the end of this project, it evolved into an example of a reply from a real estate listing agent chatbot. While there is not a full implementation of a chatbot, the foundation for it is here, so I've decided to title the project Real Estate Listing Agent.

As of before the submission deadline, the following GenAI capabilities are covered in this project:
* Structured output/JSON mode/controlled generation
* Few-shot prompting
* Image understanding
* Embeddings
* Retrieval augmented generation (RAG)
* Vector search/vector store/vector database



## Prerequisites

Include retry policy and be able to use the Google API Key.

In [ ]:
# Uninstall packages from Kaggle base image that are not needed.
!pip uninstall -qqy jupyterlab kfp
# Install the google-genai SDK for this codelab.
!pip install -qU 'google-genai==1.7.0' 'chromadb==0.6.3'

In [ ]:
from google import genai
from google.genai import types

from IPython.display import Markdown, HTML, display

genai.__version__

In [ ]:
# API KEY

from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

# Define a retry policy. The model might make multiple consecutive calls automatically
# for a complex query, this ensures the client retries if it hits quota limits.
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## Dataset

I created a small dataset using images from Unsplash. These are images related to real estate. I added it as an input into the notebook. 

In [ ]:
def upload_to_google(path):
    image = client.files.upload(file=path)
    return image

## Upload to Google

In order to send it Gemini, I must save the file to Google. Below, I define some helper methods to upload the files to Google so it can be passed to Gemini for interpretation.

In [ ]:
# delete google files, resets images stored in Google to avoid duplicates
print('List all images stored in Google via File API:')
for f in client.files.list():
    print(' ', f.name)
    client.files.delete(name=f.name)

In [ ]:
import os

# for easy mapping of image dataset for display to verify results
image_map = {}

# list all files in the dataset and upload to google
for dirname, _, filenames in os.walk('/kaggle/input/'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        image = upload_to_google(path)
        image_map[image.name] = path

print(image_map)

In [ ]:
# list files
print('List all images stored in Google via File API:')
last_file = ""
for f in client.files.list():
    last_file = f.name
    print(' ', f.name)

In [ ]:
# captions the image as a "hello world" test that it works
# this is code directly from the documentation
def process_image(image_path):
    image = client.files.get(name=image_path)
    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[image, "Caption this image."])

    print(response.text)

In [ ]:
process_image(last_file)

## Prompt Engineering Improvements

I can see that it's working now, so I can improve the prompt to get a better description of the image.

In this section, I include a few more GenAI capabiltiies: Few-shot prompting, and Structured output/JSON mode/controlled generation.

In [ ]:
prompt = """
Examine the image provided. This image is part of a real estate listing.

To enhance the listing, important details need to be identified in the image to include in the listing. 
A title is needed that includes at least 3 key attributes visible in the image that would 
make the listing marketable and appealing to a renter or a buyer.
These details should be included in the 'title' attribute in the sample json below. 
It should be followed by a 'description' attribute that contains at least 150 words and describes the photo in detail and 
highlights the key attributes and additional attributes that increase the value of the listing.

The image should also be evaluated for the presence of a pool. 

If a pool is present, the json output should include a boolean value indicating a pool is present.

The output should be in JSON.

EXAMPLE:
The image of a kitchen contains an island. A tag would be "kitchen island".
JSON Response:
```
{
"title": "A beautiful kitchen with an island in the center and a large sink."
"description": "A bright and spacious kitchen with white cabinets, a blue island, and dark countertops. The white cabinets have ample space for pots, pans, silverware, tupperware, bakeware, mixing bowls and other kitchen tools. The pendant lights brighten the kitchen as does the cathedral windows in the ceiling that allow natural light into the kitchen illuminating the countertops. The stainless steel refrigerator contains ample space.",
"tags": ["kitchen island"],
"has_pool": false
}

EXAMPLE:
The image of a house contains a pool. A tag would be "outdoor pool".
JSON Response:
```
{
"title": "A picturesque two-story house with a red exterior stands nestled between lush greenery and neighboring homes under a clear blue sky.",
"description": "This beautiful two-story home is a perfect starter home with a bright red exterior giving it a natural rustic appeal. The landscaping in the front blooms in the spring while the evergreens give it a perpertual green in the winter. The porch contains enough space for two chairs to enjoy the evening breeze and read a book with a cup of tea.",
"tags": ["kitchen island"],
"has_pool": true
}
"""
import typing_extensions as typing
import json

class ImageTags(typing.TypedDict):
    title: str
    description: str
    tags: list[str]
    has_pool: bool

def process_image_tags(image_path):
    image = client.files.get(name=image_path)
    response = client.models.generate_content(
            model="gemini-2.0-flash",
            config=types.GenerateContentConfig(
            temperature=0.5,
            response_mime_type="application/json",
            response_schema=ImageTags,
            ),
            contents=[image, prompt])

    return json.loads(response.text) | { "path": image_map[image_path] }

In [ ]:
process_image_tags(last_file)

## Embeddings and Vector Databases

Once the tags are identified, the information can be added to a database for querying. As the next step of this project, I'll test out Embeddings and RAGs.

In [ ]:
for m in client.models.list():
    if "embedContent" in m.supported_actions:
        print(m.name)

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry

from google.genai import types


# Define a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})


class GeminiEmbeddingFunction(EmbeddingFunction):
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    @retry.Retry(predicate=is_retriable)
    def __call__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        response = client.models.embed_content(
            model="models/text-embedding-004",
            contents=input,
            config=types.EmbedContentConfig(
                task_type=embedding_task,
            ),
        )
        return [e.values for e in response.embeddings]

In [ ]:
from PIL import Image

# create a list of documents to be added to the database
documents = []

print('Processing all image files with Gemini API...')
for f in client.files.list():
    document = process_image_tags(f.name)
    documents.append(document)
    print(' ', document)

In [ ]:
# display images to verify results and limit # to avoid memory problems
for idx, document in enumerate(documents): 
    if idx < 3:
        display(Image.open(document['path']))
    print(' ', document)

In [ ]:
import chromadb

DB_NAME = "real_estate_listings_db"

embed_fn = GeminiEmbeddingFunction()
embed_fn.document_mode = True

chroma_client = chromadb.Client()
# chroma_client.delete_collection(name=DB_NAME) # reset db

db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)


In [ ]:
for document in documents:
    document_text = "Title: " + document['title'] + "\nDescription: " + document['description']
    print(document_text)
    db.add(
        documents=[document['title']],
        metadatas=[{"tags": ','.join(document['tags']), "has_pool": document['has_pool'], "description": document['description']}],
        ids=[document['path']]
    )

In [ ]:
db.count()

## Query Database

After adding embeddings and documents to database, now the database can be queried.

In [ ]:
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "Show me a listing with a red exterior."

result = db.query(query_texts=[query], n_results=1)

display(Image.open(result['ids'][0][0]))


[all_passages] = result["documents"]

Markdown(all_passages[0])

In [ ]:
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "Show me a two-story home."

result = db.query(query_texts=[query], n_results=1)

display(Image.open(result['ids'][0][0]))


[all_passages] = result["documents"]

Markdown(all_passages[0])

In [ ]:
# Search the Chroma DB using the specified query.
query = "Show me listings with a pool."

result = db.query(query_texts=[query], n_results=3)

display(Image.open(result['ids'][0][0]))

[all_passages] = result["documents"]

print("Distances: ", result["distances"])

Markdown(all_passages[0])

## RAGs

After querying the database, it augments the prompt with the results and generates a "final answer".

In [ ]:
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative real estate listing agent bot that answers questions using information 
from the listings included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
Strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
"""

# Add the retrieved documents to the prompt.
for idx, passage in enumerate(all_passages):
    passage_oneline = passage.replace("\n", " ")
    prompt += f"Listing {idx + 1}: {passage_oneline} \n"

print(prompt)

In [ ]:
answer = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=prompt)

Markdown(answer.text)

## Next Steps

After all this experimentation, there were definitely areas I would need to continue to work more on to make it useful.  As a larger project, I could see how this work could be extended to build a real estate agent chat bot that could help personalize the property renting or buying experience. 

There were definitely some challenges I came across as I worked on different aspects of this project though. For example, there were some iterations of titles and descriptions for the images where the results were hallucinations of the actual image. In one particular image, it described it as a "waterfront" home, when there was really no evidence of the home being a "waterfront" home (i.e. no beaches, lakes, etc.) These hallucinations would need to be addressed through negative prompting or fine-tuning.

I also think the results could be quite interesting if allowed to interpret multiple images as a collective set so it could make sense of multiple images as a single property listing. In this project, I mostly experimented with each image as an individual listing for simplicity sake. I would certainly be interested in experimenting with images as collections for a single property to see what could be accomplished with more content.